
# Gráfica y representación de la consulta 1

En esta sección, se presenta la gráfica de barras y lineal generada a partir de los datos obtenidos de la consulta 1. La consulta selecciona información relevante sobre las canciones y su movimiento diario y semanal en diferentes países. La gráfica muestra el movimiento diario por canción y país, utilizando la biblioteca Plotly para una visualización interactiva.

In [ ]:
# Importar librerías
import dash
import psycopg2
from dash import Dash, html, dcc
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import pandas as pd

try:
    # Conexión a la base de datos
    connection = psycopg2.connect(
        host='localhost',
        user='postgres',  # Reemplaza con tu usuario de la base de datos
        password='2I@@`7ao*3}*',  # Reemplaza con tu contraseña
        database='proyecto'
    )
    print("Conexión exitosa")
    
    # Crear un cursor
    cursor = connection.cursor()
    
    # Ejecutar la consulta
    t_consulta1 = "SELECT c.nombre AS cancion, r.pais_codigo, r.ranking_diario, r.movimiento_diario, r.movimiento_semanal, c.danceability, c.energy, c.tempo FROM Rankings r JOIN Canciones c ON r.spotify_id = c.spotify_id WHERE r.movimiento_diario IS NOT NULL AND r.movimiento_diario > 0 OR r.movimiento_semanal IS NOT NULL AND r.movimiento_semanal > 0 ORDER BY r.movimiento_diario DESC, r.movimiento_semanal DESC;"

    cursor.execute(t_consulta1)
    rows = cursor.fetchall()
    
    # Obtener los nombres de las columnas
    column_names = [desc[0] for desc in cursor.description]
    
    # Crear un DataFrame
    df = pd.DataFrame(rows, columns=column_names)
    print(df)
    
    # Generar una gráfica con Plotly
    fig = px.bar(
        df,
        x='cancion',
        y='movimiento_diario',
        color='pais_codigo',
        title='Movimiento Diario por Canción y País',
        labels={
            'movimiento_diario': 'Movimiento Diario', 
            'cancion': 'Canción', 
            'pais_codigo': 'País'
        }
    )
    
    # Mostrar la gráfica
    fig.show()
    
except Exception as error:
    print("Error al conectar o ejecutar la consulta:", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Conexión cerrada")

In [1]:
# Importar librerías
try:
    # Conexión a la base de datos
    connection = psycopg2.connect(
        host='localhost',
        user='postgres',  # Reemplaza con tu usuario de la base de datos
        password='2I@@`7ao*3}*',  # Reemplaza con tu contraseña
        database='proyecto'
    )
    print("Conexión exitosa")
    
    # Crear un cursor
    cursor = connection.cursor()
    
    # Ejecutar la consulta
    t_consulta1 = "SELECT c.nombre AS cancion, r.pais_codigo, r.ranking_diario, r.movimiento_diario, r.movimiento_semanal, c.danceability, c.energy, c.tempo FROM Rankings r JOIN Canciones c ON r.spotify_id = c.spotify_id WHERE r.movimiento_diario IS NOT NULL AND r.movimiento_diario > 0 OR r.movimiento_semanal IS NOT NULL AND r.movimiento_semanal > 0 ORDER BY r.movimiento_diario DESC, r.movimiento_semanal DESC;"

    cursor.execute(t_consulta1)
    rows = cursor.fetchall()
    
    # Obtener los nombres de las columnas
    column_names = [desc[0] for desc in cursor.description]
    
    # Crear un DataFrame
    df = pd.DataFrame(rows, columns=column_names)
    print(df)
    
    # Generar una gráfica con Plotly
    fig = px.bar(
        df,
        x='cancion',
        y='movimiento_diario',
        color='pais_codigo',
        title='Movimiento Diario por Canción y País',
        labels={
            'movimiento_diario': 'Movimiento Diario', 
            'cancion': 'Canción', 
            'pais_codigo': 'País'
        }
    )
    
    # Mostrar la gráfica
    fig.show()
    
except Exception as error:
    print("Error al conectar o ejecutar la consulta:", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Conexión cerrada")

Error al conectar o ejecutar la consulta: name 'psycopg2' is not defined


NameError: name 'connection' is not defined